In [8]:
import itertools
import torch
import pickle

In [51]:
class R:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in range(3)))
        self.n_hat = {combination:self.make_NN() for combination in combination_list}
   
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def predict(self, weights, theta):
        term1=torch.sum( torch.stack( [(1. + weights[c]*theta)**2 for c in self.coefficient_names ]), dim=0)
        term2=torch.sum( torch.stack( [torch.sum( torch.stack( [ predictions[tuple(sorted((c_1,c_2)))]*eft[c_2] for c_1 in self.coefficients[i_c_2:] ]), dim=0)**2 for i_c_2, c_2 in enumerate(self.coefficients) ] ), dim=0)
        return torch.add(term1,term2)  
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [ ]:
#if __name__=='__main__':
r = R(5, ['cHW', 'cHWtil'])
r.save('test')
r2=R.load('test')

NameError: name 'combination_list' is not defined